INGESTÃO DE DADOS

In [ ]:
import pandas as pd


# Carregar datasets
file_paths = {
    'admissions': 'csvs/ADMISSIONS.csv',
    'microbiology_events': 'csvs/MICROBIOLOGYEVENTS.csv',
    'prescriptions': 'csvs/PRESCRIPTIONS.csv',
    'labevents': 'csvs/LABEVENTS.csv',
    'diagnoses_icd': 'csvs/DIAGNOSES_ICD.csv',
    'procedures_icd': 'csvs/PROCEDURES_ICD.csv',
    'patients': 'csvs/PATIENTS.csv',
    'icustays': 'csvs/ICUSTAYS.csv'
}
files = {}
for name, path in file_paths.items():
    try:
        files[name] = pd.read_csv(path)
    except FileNotFoundError:
        print(f"File not found: {path}")


# Filtrar dados relevantes
microbiologia_filtered = files["microbiology_events"][['row_id', 'subject_id', 'hadm_id', 'chartdate', 'charttime', 
           'spec_itemid', 'spec_type_desc', 'org_itemid', 'org_name', 
           'isolate_num', 'ab_itemid', 'ab_name', 'dilution_text', 
           'dilution_comparison', 'dilution_value', 'interpretation']] #done

prescricoes_filtered = files["prescriptions"][['subject_id', 'drug', 'enddate', ]]

admicoes_filtered = files["admissions"][['subject_id', 'admittime', 'dischtime', 'ethnicity', 'admission_type']] #done

diagnosticos_filtered = files["diagnoses_icd"][['subject_id', 'icd9_code', 'seq_num']] #done

eventoslab_filtered = files["labevents"][['subject_id', 'itemid', 'valuenum', 'valueuom', 'flag']] #done

procedimentos_filtered = files["procedures_icd"][['subject_id', 'icd9_code', 'seq_num']] #done

pacientes_filtered = files["patients"][['subject_id', 'gender', 'dob', 'dod']] #done?

estadias_filtered = files["icustays"][['subject_id', 'los', 'first_careunit', 'last_careunit']] #done

# Realizar merge com base no subject_id 
merged_data = microbiologia_filtered.merge(prescricoes_filtered, on='subject_id', how='outer')
merged_data = merged_data.merge(admicoes_filtered, on='subject_id', how='outer')
merged_data = merged_data.merge(diagnosticos_filtered, on='subject_id', how='outer')
merged_data = merged_data.merge(eventoslab_filtered, on='subject_id', how='outer')
merged_data = merged_data.merge(procedimentos_filtered, on='subject_id', how='outer')
merged_data = merged_data.merge(pacientes_filtered, on='subject_id', how='outer')
merged_data = merged_data.merge(estadias_filtered, on='subject_id', how='outer')
#converter para dataframe, e tratar dados
merged_data_df = pd.DataFrame(merged_data)
merged_data_df.drop_duplicates()
merged_data_df.dropna()





RESISTENCIA

In [ ]:
# Identificar padrões de resistência
resistance = merged_data[merged_data['interpretation'] == 'R']
microbiologia_filtered = resistance[['ab_name', 'interpretation']]
resistance = microbiologia_filtered.groupby(['ab_name', 'interpretation']).size().reset_index(name='counts')
print(resistance)

SUGESTAO 

In [66]:
def drug_suggestion(org_name, resistance):
    if resistance == 'R':
        return "Sugestão de antibiótico alternativo"
    else:
        return "Sugestão de antibiótico padrão"

merged_data1['sugestao'] = merged_data1.apply(lambda x: drug_suggestion(x['org_name'], x['interpretation']), axis=1)


FERRAMENTA VALIDAÇÃO

In [ ]:
from sklearn.metrics import accuracy_score

# Testar a ferramenta em um conjunto de dados de validação
resultados_reais = merged_data1['antibiotico_real']
sugestoes = merged_data1['sugestao']

acuracia = accuracy_score(resultados_reais, sugestoes)
print(f"Acurácia da ferramenta: {acuracia}")
